In [1]:
__author__ = '@Tssp'
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from codvidutils.Autoencoder_Transformation_main import Transformation_main
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize

ImportError: No module named 'codvidutils'

# Prepare dataset

In [ ]:
outputs = Transformation_main('data/train_split_v4.csv', 'data/test_split_v4.csv')
Y_test = outputs['Y_test']
encoder_train = outputs['encoder_train']
encoder_test = outputs['encoder_test']
del outputs

# XGBoost

## Regressor:

In [ ]:
predsr = np.loadtxt('log/preds_XGBr_lr0.01_n250.txt', delimiter=',')
nocovid = predsr[np.where(Y_test == 0)]
covid = predsr[np.where(Y_test == 1)]

In [ ]:
TP_050 = np.count_nonzero(np.where((Y_test==0) & (predsr<0.50)))
FN_050 = np.count_nonzero(np.where((Y_test==1) & (predsr>0.50)))
FP_050 = np.count_nonzero(np.where((Y_test==1) & (predsr<0.50)))
TN_050 = np.count_nonzero(np.where((Y_test==0) & (predsr>0.50)))
cm = np.array([[TP_050, TN_050],[FN_050, FP_050]])
print('{} of {} no COVID-19 ({:2.3}%)'.format(cm[0,0], np.sum(Y_test == 0), 100*cm[0, 0]/np.sum(Y_test == 0)))
print('{} of {} COVID-19 ({:2.3}%)'.format(cm[1,1], np.sum(Y_test == 1), 100*cm[1,1]/np.sum(Y_test == 1)))
print(cm)
cm = normalize(cm, 'l1')
labels = ['no COVID-19', 'COVID-19']
fig1, ax = plt.subplots(figsize=(8, 6))
cax = ax.matshow(cm, cmap='YlGnBu', vmin=0, vmax=1)
cbar = fig1.colorbar(cax)
cbar.set_ticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])
cbar.set_ticklabels(['0', '0.2', '0.4', '0.6', '0.8', '1.0'])
cbar.ax.set_ylabel('Accuracy', rotation=270, labelpad=20, fontsize=16)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
ax.set_ylabel('Expected', fontsize=16)
ax.set_xlabel('Predicted', fontsize=16)
fig1.savefig('Figures/cm_XGBregressor_lr001_n250.png', dpi=200)

In [ ]:
fig2, ax = plt.subplots(figsize=(8,6))
ax.hist(nocovid, 50, histtype='step', lw=2, color='darkorange', label='No COVID')
ax.hist(covid, 50, histtype='step', lw=2, color='darkblue', label='COVID')
plt.grid()
ax.set_ylabel('Number of events /0.01', fontsize=16)
ax.set_xlabel('Score', fontsize=16)
plt.legend( loc='upper right', fontsize=14)
plt.yscale("log")
fig2.savefig('Figures/histogram_XGBregressor_lr001_n250.png', dpi=200)